# 스트리밍 (Streaming)

LangChain의 **스트리밍 시스템(Streaming system)** 은
에이전트 실행(agent run) 중 발생하는 **실시간 피드백(live feedback)** 을
애플리케이션으로 직접 전달할 수 있게 해줍니다.

스트리밍을 사용하면 **전체 응답이 완성되기 전에** 부분적으로 결과를 점진적으로 표시할 수 있습니다.
이를 통해 **LLM의 지연(latency)** 으로 인한 대기 시간을 줄이고,
사용자에게 더 빠르고 자연스러운 **실시간 인터랙션 경험(UX)** 을 제공합니다.

### LangChain 스트리밍으로 가능한 기능들

* **에이전트 진행 상황 스트리밍 (Stream agent progress)**
  → 각 에이전트 단계가 끝날 때마다 상태 업데이트(state update)를 수신할 수 있습니다.

* **LLM 토큰 스트리밍 (Stream LLM tokens)**
  → 언어 모델이 토큰을 생성할 때마다, 생성되는 즉시 스트리밍으로 받아볼 수 있습니다.

* **사용자 정의 업데이트 스트리밍 (Stream custom updates)**
  → 예를 들어 `"100개 중 10개 레코드 가져옴"` 같은
  사용자 정의 신호(user-defined signals)를 실시간으로 전송할 수 있습니다.

* **다중 모드 스트리밍 (Stream multiple modes)**
  → 다음과 같은 모드 중 선택적으로 사용할 수 있습니다:

  1. **updates** – 에이전트 진행 상황 중심의 업데이트
  2. **messages** – LLM 토큰 및 관련 메타데이터
  3. **custom** – 임의의 사용자 정의 데이터 (예: 로그, 상태 메시지 등)
  4. **values** - LLM의 최종 응답 객체들을 순서대로 반환

| 모드             | 무엇을 스트리밍하나                                          | 포함 이벤트                        | 언제 쓰면 좋은가               |
| -------------- | --------------------------------------------------- | ----------------------------- | ----------------------- |
| **`updates`**  | 에이전트 진행 단계 업데이트 (각 노드·도구 호출의 상태 변화, 요약 메시지)         | 노드 진입/종료, 도구 호출·결과 요약, 라우팅 결정 | 단계별 진행률 표시, 로그, UI 타임라인 |
| **`messages`** | LLM이 생성하는 토큰 또는 문장                                  | 모델 출력 토큰, 도구 호출 중간 메시지        | 실시간 typing 효과, 스트리밍 응답 표시   |
| **`custom`**   | 도구 내부의 사용자 정의 업데이트  | 도구 진행 상태, 중간 결과, 사용자 로그       | 데이터 수집·ETL·크롤링 진행 상황 표시 |
| **`values`**   | LLM의 최종 응답 객체 (`AIMessageChunk` 시퀀스)                | 최종 응답 조각 → 누적 결합 가능           | 청크를 모아 후처리하거나 완성 메시지 생성 |


In [ ]:
# model = init_chat_model("gpt-5-nano", model_provider="openai")

### 에이전트 진행 상황 스트리밍 (Stream agent progress)
각 에이전트 단계가 끝날 때마다 상태 업데이트(state update)를 수신할 수 있습니다.

In [ ]:
def get_weather(city: str) -> str:
# 에이전트 생성
# 스트리밍 실행

### LLM 토큰 (LLM tokens)

LLM이 **토큰을 생성하는 즉시 실시간으로 스트리밍**하려면
`stream_mode="messages"` 옵션을 사용하면 됩니다.

이 모드를 사용하면, **에이전트가 실행 중 호출하는 도구(tool)들의 스트리밍 출력**과
**최종 응답(final response)** 을 모두 실시간으로 확인할 수 있습니다.

In [ ]:
def get_weather(city: str) -> str:
# 에이전트 생성
# LLM이 토큰을 생성할 때마다 실시간으로 스트리밍 받기

### 사용자 정의 업데이트 (Custom updates)

도구(tool)가 실행되는 동안 실시간으로 상태 업데이트를 스트리밍(streaming) 하려면
`get_stream_writer`를 사용할 수 있습니다.

즉, 도구가 실행되는 과정에서 중간 진행 상황이나 결과를 실시간으로 전송할 수 있게 하는 기능입니다.
이 기능을 사용하면, 모델의 최종 응답을 기다리지 않고도 사용자는 즉시 진행 상태를 확인할 수 있습니다.

In [ ]:
def get_weather(city: str) -> str:
    # 임의의 데이터를 스트리밍 방식으로 출력
# 에이전트 생성
# 사용자 질의에 대해 커스텀 스트리밍 실행

## 스트리밍과 청크 (Streaming and Chunks)

스트리밍 중에는 **`AIMessageChunk` 객체**들이 순차적으로 전달되며,
이 청크(chunk)들을 결합하면 **완전한 메시지 객체(full message object)** 를 구성할 수 있습니다.


In [ ]:
# 모델로부터 스트리밍 방식으로 응답을 받음 (stream_mode=values)
    # full_message가 비어있다면 첫 청크를 넣고, 이후에는 청크를 이어붙임
# 모든 청크를 합친 최종 메시지 출력